# Hygge-route planner

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import contextily as cx
import osmnx as ox
import networkx as nx

import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

from collections import Counter
from random import choice

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


## Load data
This data is exported by the `birds.ipynb` notebook. Birds are attributed to an edge if a 50m buffer around the bird intersects with a given street. Check out the notebook for more details.

In [64]:
edges = gpd.read_file('../data/edges_birds.json')
edges = edges.set_crs('EPSG:25832', allow_override=True)
edges.set_index(edges.id.apply(lambda x: x[1:-1].split(',')))

,id,access,bird_counts,birds_per_m,junction,length,oneway,geometry
id,,,,,,,,
"[118725, 298795432, 0]","(118725, 298795432, 0)",None,8.0,0.070877,None,112.872,True,"LINESTRING (724025.025 6175551.339, 724031.889..."
"[118725, 6357644306, 0]","(118725, 6357644306, 0)",None,3.0,0.074080,None,40.497,True,"LINESTRING (724025.025 6175551.339, 724051.956..."
"[298795432, 298795496, 0]","(298795432, 298795496, 0)",None,7.0,0.077950,None,89.801,True,"LINESTRING (724075.369 6175450.104, 724076.732..."
"[6357644306, 2378383724, 0]","(6357644306, 2378383724, 0)",None,2.0,0.091199,None,21.930,True,"LINESTRING (724059.919 6175572.112, 724075.704..."
"[6357644306, 6357644350, 0]","(6357644306, 6357644350, 0)",None,1.0,0.043373,None,23.056,True,"LINESTRING (724059.919 6175572.112, 724081.508..."
...,...,...,...,...,...,...,...,...
"[11808938472, 11808938473, 0]","(11808938472, 11808938473, 0)",None,0.0,0.000000,None,13.119,False,"LINESTRING (722920.161 6172629.725, 722915.850..."
"[11808938474, 11808938475, 0]","(11808938474, 11808938475, 0)",None,0.0,0.000000,None,10.142,False,"LINESTRING (722930.119 6172636.344, 722927.113..."
"[11808951459, 11808951460, 0]","(11808951459, 11808951460, 0)",None,5.0,0.221582,None,22.565,False,"LINESTRING (722651.508 6180338.032, 722673.654..."


In [65]:
edges.head()

,id,access,bird_counts,birds_per_m,junction,length,oneway,geometry
0,"(118725, 298795432, 0)",None,8.0,0.070877,None,112.872,True,"LINESTRING (724025.025 6175551.339, 724031.889..."
1,"(118725, 6357644306, 0)",None,3.0,0.074080,None,40.497,True,"LINESTRING (724025.025 6175551.339, 724051.956..."
2,"(298795432, 298795496, 0)",None,7.0,0.077950,None,89.801,True,"LINESTRING (724075.369 6175450.104, 724076.732..."
3,"(6357644306, 2378383724, 0)",None,2.0,0.091199,None,21.930,True,"LINESTRING (724059.919 6175572.112, 724075.704..."
4,"(6357644306, 6357644350, 0)",None,1.0,0.043373,None,23.056,True,"LINESTRING (724059.919 6175572.112, 724081.508..."


In [46]:
nodes = gpd.read_file('../data/nodes.json').to_crs('EPSG:25832')
nodes = nodes.set_index('id')

In [47]:
nodes.head()

,highway,street_count,x,y,geometry
id,,,,,
118725,None,3,12.562906,55.673985,POINT (724025.025 6175551.339)
118732,traffic_signals,4,12.561510,55.663099,POINT (723999.539 6174336.199)
118735,None,4,12.571933,55.668238,POINT (724625.288 6174941.454)
118738,None,3,12.561835,55.662837,POINT (724021.470 6174308.056)
118744,None,4,12.560034,55.661718,POINT (723914.631 6174177.779)


## Find best path minimizing distance and maximizing birds

### Weighted sum of features
One approach to create a routing algorithm that maximizes "hygge" is to weight each feature, and find the shortest path on this new composite feature. We use minmax scaling to get our features down into a comparable range
$$
x_{scaled} = \frac{x-x_{min}}{x_{max}-x_{min}}
$$
For the birds, we use $1-x_{scaled}$ as the feature, such that we can minimize the feature to maximize bird sightings

In [25]:
edges['w1'] = (edges['length'] - edges['length'].min()) / (edges['length'].max() - edges['length'].min())
edges['w2'] = 1 - ((edges['bird_counts'] - edges['bird_counts'].min()) / (edges['bird_counts'].max() - edges['bird_counts'].min()))

We can now create a "hygge" score as a weighted sum of the relevant features. The specific weights are parameters that need to be tuned for optimal routing.

In [27]:
edges['w'] = 0.9 * edges['w1'] + 0.1 * edges['w2']

We create a graph with the new hygge score on each edge.

In [66]:
G = ox.graph_from_gdfs(nodes, edges)

/opt/conda/lib/python3.9/site-packages/osmnx/utils_graph.py:155: UserWarning: discarding the gdf_nodes geometry column, though its values differ from the coordinates in the x and y columns
  warnings.warn(


TypeError: cannot unpack non-iterable int object

This new hygge score is designed to be minimized. By using traditional shortest path algorithms, we can find the path that minimizes the hygge score, which in turn leads to minimizing path length and maximizing bird sightings.

### Multiobjective shortest path - MOSP